# Machine Learning для аналитика — единый конспект

В этом ноутбуке собраны основные темы ML для аналитика:
1) **Основы ML** (train/test, кодирование, масштабирование, базовые метрики)  
2) **Регрессия** (линейная/множественная, полиномиальная, метрики)  
3) **Классификация** (логистическая регрессия, дерево решений, Random Forest, метрики)  
4) **Кластеризация** (k-means, иерархическая, DBSCAN, Elbow, Silhouette)  
5) **Бустинг и ансамбли** (идея, Gradient Boosting; XGBoost/LightGBM/CatBoost — опционально)

⚙️ Подход: максимум понятности, минимум магии. Код снабжён комментариями **к каждой строке**.

---

## 1. Основы ML: подготовка данных и базовая модель
**Что покажем:**
- Разделение данных на train/test с `stratify`
- Кодирование категориальных признаков (One-Hot)
- Масштабирование числовых признаков (StandardScaler)
- Пайплайн (Pipeline) = предобработка + модель
- Метрики классификации (Accuracy/Precision/Recall/F1)

Мы создадим маленький искусственный датасет: `city` (категория), `age` (число), `bought` (класс 0/1).

In [3]:
import pandas as pd  # импортируем pandas для работы с таблицами
from sklearn.model_selection import train_test_split  # функция для разбиения на train/test
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # кодирование категориальных, масштабирование числовых
from sklearn.compose import ColumnTransformer  # позволяет применять разные трансформации к разным колонкам
from sklearn.pipeline import Pipeline  # объединяет шаги предобработки и модель в одну цепочку
from sklearn.linear_model import LogisticRegression  # базовая модель бинарной классификации
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # метрики качества

# создаём небольшой датафрейм с категориальным и числовым признаком и целевой переменной
df = pd.DataFrame({
    'city': ['Moscow', 'SPB', 'Kazan', 'Moscow', 'Kazan', 'SPB', 'Moscow', 'SPB'],  # города (категория)
    'age': [22, 35, 41, 28, 50, 31, 23, 47],  # возраст (число)
    'bought': [0, 1, 1, 0, 1, 0, 0, 1]  # целевой класс: купил (1) / нет (0)
})

X = df[['city', 'age']]  # матрица признаков: берём столбцы city и age
y = df['bought']  # целевая переменная (Series)

cat_features = ['city']  # список категориальных колонок
num_features = ['age']   # список числовых колонок

# ColumnTransformer применяет One-Hot к city и StandardScaler к age
preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),  # OHE: векторизуем город
        ('num', StandardScaler(), num_features)  # стандартизируем возраст (ср.=0, std=1)
    ]
)

# Pipeline: сначала preprocess, потом обучаем LogisticRegression
clf = Pipeline(steps=[
    ('prep', preprocess),  # шаг предобработки признаков
    ('model', LogisticRegression(max_iter=1000, random_state=42))  # сама модель
])

# train_test_split с stratify=y — распределение классов сохраняется и в train, и в test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y  # 25% данных в тест, фиксируем случайность и стратифицируем
)

clf.fit(X_train, y_train)  # обучаем пайплайн на обучающих данных
y_pred = clf.predict(X_test)  # делаем предсказания классов на тесте

print('Accuracy:', accuracy_score(y_test, y_pred))   # доля верных ответов
print('Precision:', precision_score(y_test, y_pred)) # сколько из предсказанных 1 реально 1
print('Recall:', recall_score(y_test, y_pred))       # какую долю всех реальных 1 нашли
print('F1:', f1_score(y_test, y_pred))               # гармоническое среднее precision и recall


Accuracy: 0.5
Precision: 0.5
Recall: 1.0
F1: 0.6666666666666666


### Задания (Основы)
1. Добавьте в датафрейм новый категориальный признак (например, `channel` = ['online','offline']) и обновите пайплайн.  
2. Поменяйте тестовый размер на 0.4 и посмотрите, как меняются метрики.  
3. Попробуйте удалить `StandardScaler` — изменится ли качество? Почему?  
4. Добавьте в `LogisticRegression` параметр `C` (например, 0.5 и 2.0) и сравните метрики.  
5. Выведите `clf.named_steps['prep'].get_feature_names_out()` и посмотрите, какие фичи получились после OHE.

## 2. Регрессия: линейная и полиномиальная
**Что покажем:**
- Линейная регрессия на реальном датасете `load_diabetes`
- Полиномиальная регрессия на синтетических данных
- Метрики: MAE, MSE, R²

Почему `load_diabetes`? Это готовый датасет из sklearn, не требует загрузки из интернета и часто используется как учебный пример для регрессии.

In [4]:
import numpy as np  # для численных операций
from sklearn.datasets import load_diabetes  # датасет для регрессии (прогноз показателя болезни)
from sklearn.linear_model import LinearRegression  # линейная регрессия
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # метрики регрессии
from sklearn.model_selection import train_test_split  # разбиение на train/test
from sklearn.preprocessing import StandardScaler  # масштабирование признаков
from sklearn.pipeline import Pipeline  # конвейер из шагов

diab = load_diabetes()  # загружаем датасет (X — признаки, y — целевая числовая)
X = diab.data           # матрица признаков (уже числовые, стандартизованные диапазоны)
y = diab.target         # целевая переменная (число)

# разбиение 80/20, фиксируем random_state для воспроизводимости
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# конвейер: масштабирование (дополнительно) + линейная регрессия
reg = Pipeline(steps=[
    ('scaler', StandardScaler()),  # масштабируем признаки
    ('linreg', LinearRegression()) # обучаем линейную регрессию
])

reg.fit(X_train, y_train)   # обучаем модель на train
y_pred = reg.predict(X_test) # предсказываем для test

print('MAE:', mean_absolute_error(y_test, y_pred))  # средняя абсолютная ошибка (в ед. y)
print('MSE:', mean_squared_error(y_test, y_pred))   # средний квадрат ошибки (чувствителен к выбросам)
print('R^2:', r2_score(y_test, y_pred))             # доля объяснённой дисперсии (чем ближе к 1, тем лучше)


MAE: 42.794094679599944
MSE: 2900.1936284934827
R^2: 0.45260276297191926


### Полиномиальная регрессия (синтетические данные)
Смоделируем зависимость \(y = 0.5x^2 + 1.0x + 2.0 + \text{шум}\). Используем `PolynomialFeatures(degree=2)` для добавления \(x^2\).

In [5]:
from sklearn.preprocessing import PolynomialFeatures  # генерация полиномиальных признаков

rng = np.random.default_rng(42)          # генератор случайностей для воспроизводимости
x = np.linspace(-3, 3, 150).reshape(-1, 1)  # создаём столбец значений x от -3 до 3
y_true = 0.5 * x[:, 0]**2 + 1.0 * x[:, 0] + 2.0  # истинная квадратичная зависимость (без шума)
y = y_true + rng.normal(0, 0.5, size=len(x))     # добавляем шум с нормальным распределением

poly_reg = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # генерируем признаки x и x^2
    ('linreg', LinearRegression())                               # обучаем линейную регрессию на расширенных фичах
])

poly_reg.fit(x, y)               # обучаем модель на всех точках (для простоты)
y_pred_poly = poly_reg.predict(x) # предсказываем на тех же x (оценим, как модель подстроилась)

print('MAE (poly):', mean_absolute_error(y, y_pred_poly))  # метрики ошибки
print('MSE (poly):', mean_squared_error(y, y_pred_poly))
print('R^2 (poly):', r2_score(y, y_pred_poly))


MAE (poly): 0.3426751184738956
MSE (poly): 0.1809042206645824
R^2 (poly): 0.9640457572787543


### Задания (Регрессия)
1. Попробуйте в `PolynomialFeatures` поменять `degree` на 3 и 4. Что происходит с R²?  
2. В `load_diabetes` уберите `StandardScaler`. Изменятся ли метрики? Почему?  
3. Добавьте в линейную регрессию регуляризацию: замените `LinearRegression` на `Ridge` (или `Lasso`).  
4. Посчитайте `RMSE = sqrt(MSE)` и сравните с `MAE`. Чем эти метрики отличаются по смыслу?  
5. Разбейте синтетические данные на train/test (например, 70/30) и сравните качество на train и test (переобучение?).

## 3. Классификация: логистическая регрессия, дерево, случайный лес
**Что покажем:**
- Датасет `load_breast_cancer` (бинарная классификация)
- Три модели: Logistic Regression, Decision Tree, Random Forest
- Метрики: Accuracy, Precision, Recall, F1, ROC-AUC

Важно: используем `stratify=y` при разбиении, чтобы классы были представлены корректно и в train, и в test.

In [6]:
from sklearn.datasets import load_breast_cancer  # встроенный датасет для бинарной классификации
from sklearn.tree import DecisionTreeClassifier   # дерево решений
from sklearn.ensemble import RandomForestClassifier  # случайный лес (ансамбль деревьев)
from sklearn.linear_model import LogisticRegression  # логистическая регрессия
from sklearn.metrics import roc_auc_score, classification_report  # ROC-AUC и текстовый отчёт по классам

data = load_breast_cancer()  # загружаем признаки и цель
X = data.data                # матрица числовых признаков
y = data.target              # целевая (0/1)

# делим на train/test с сохранением баланса классов
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 1) Логистическая регрессия с масштабированием (часто помогает для LR)
logreg = Pipeline(steps=[
    ('scaler', StandardScaler()),  # масштабируем фичи
    ('clf', LogisticRegression(max_iter=1000, random_state=42))  # обучаем LR
])
logreg.fit(X_train, y_train)               # обучение
pred_lr = logreg.predict(X_test)           # предсказания классов
proba_lr = logreg.predict_proba(X_test)[:, 1]  # вероятности класса 1 (нужны для ROC-AUC)

print('--- Logistic Regression ---')
print('Accuracy:', accuracy_score(y_test, pred_lr))
print('Precision:', precision_score(y_test, pred_lr))
print('Recall:', recall_score(y_test, pred_lr))
print('F1:', f1_score(y_test, pred_lr))
print('ROC-AUC:', roc_auc_score(y_test, proba_lr))

# 2) Дерево решений
tree = DecisionTreeClassifier(max_depth=5, random_state=42)  # ограничим глубину для устойчивости
tree.fit(X_train, y_train)               # обучение
pred_tree = tree.predict(X_test)         # предсказания классов
proba_tree = tree.predict_proba(X_test)[:, 1]  # вероятности класса 1

print('\n--- Decision Tree ---')
print('Accuracy:', accuracy_score(y_test, pred_tree))
print('Precision:', precision_score(y_test, pred_tree))
print('Recall:', recall_score(y_test, pred_tree))
print('F1:', f1_score(y_test, pred_tree))
print('ROC-AUC:', roc_auc_score(y_test, proba_tree))

# 3) Случайный лес
rf = RandomForestClassifier(n_estimators=200, random_state=42)  # ансамбль из 200 деревьев
rf.fit(X_train, y_train)                 # обучение
pred_rf = rf.predict(X_test)             # предсказания классов
proba_rf = rf.predict_proba(X_test)[:, 1]  # вероятности класса 1

print('\n--- Random Forest ---')
print('Accuracy:', accuracy_score(y_test, pred_rf))
print('Precision:', precision_score(y_test, pred_rf))
print('Recall:', recall_score(y_test, pred_rf))
print('F1:', f1_score(y_test, pred_rf))
print('ROC-AUC:', roc_auc_score(y_test, proba_rf))

print('\nClassification report (RF):\n', classification_report(y_test, pred_rf, target_names=data.target_names))  # подробный отчёт


--- Logistic Regression ---
Accuracy: 0.9824561403508771
Precision: 0.9861111111111112
Recall: 0.9861111111111112
F1: 0.9861111111111112
ROC-AUC: 0.9953703703703703

--- Decision Tree ---
Accuracy: 0.9210526315789473
Precision: 0.9565217391304348
Recall: 0.9166666666666666
F1: 0.9361702127659575
ROC-AUC: 0.9163359788359788

--- Random Forest ---
Accuracy: 0.956140350877193
Precision: 0.958904109589041
Recall: 0.9722222222222222
F1: 0.9655172413793104
ROC-AUC: 0.9930555555555556

Classification report (RF):
               precision    recall  f1-score   support

   malignant       0.95      0.93      0.94        42
      benign       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



### Задания (Классификация)
1. В `DecisionTreeClassifier` измените `max_depth` (например, 3 и 10) и сравните метрики.  
2. В `RandomForestClassifier` измените `n_estimators` (например, 50 и 500). Как это влияет на качество и время?  
3. В `LogisticRegression` попробуйте `C=0.5` и `C=2.0`. Как изменятся Precision/Recall?  
4. Посчитайте `roc_auc_score` для дерева и леса и сравните с LR.  
5. Попробуйте `class_weight='balanced'` в одной из моделей и посмотрите, как меняются метрики.

## 4. Кластеризация: k-means, иерархическая, DBSCAN
**Что покажем:**
- Синтетические данные с 3 кластерами (`make_blobs`)
- KMeans, AgglomerativeClustering, DBSCAN
- Как выбрать число кластеров: Elbow (WCSS) и Silhouette

Вместо графиков выведем численные значения (под визуализацию у тебя уже есть свой ноут).

In [7]:
from sklearn.datasets import make_blobs  # генерация синтетических кластеров
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN  # алгоритмы кластеризации
from sklearn.metrics import silhouette_score  # коэффициент силуэта для оценки качества кластеров

X, _ = make_blobs(n_samples=300, centers=3, cluster_std=1.0, random_state=42)  # генерируем 300 точек в 3 кластерах

# 4.1 KMeans (k=3)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)  # задаём k=3 и число инициализаций
labels_km = kmeans.fit_predict(X)  # обучаем и получаем метки кластеров сразу
sil_km = silhouette_score(X, labels_km)  # считаем силуэт для оценки разделения
print('KMeans: silhouette (k=3) =', sil_km)

# 4.2 Иерархическая кластеризация (агломеративная)
agg = AgglomerativeClustering(n_clusters=3)  # тоже хотим 3 кластера
labels_agg = agg.fit_predict(X)             # обучаем и получаем метки
sil_agg = silhouette_score(X, labels_agg)   # силуэт
print('Agglomerative: silhouette (k=3) =', sil_agg)

# 4.3 DBSCAN (по плотности)
db = DBSCAN(eps=0.9, min_samples=5)  # радиус соседства и минимум точек в окрестности
labels_db = db.fit_predict(X)        # метки кластеров, -1 означает шум/выбросы

# силуэт для DBSCAN считаем, только если получилось >1 кластера и не все точки шум
unique_db = set(labels_db)
if len(unique_db - {-1}) >= 2:  # как минимум 2 реальных кластера
    sil_db = silhouette_score(X, labels_db)
    print('DBSCAN: silhouette =', sil_db)
else:
    print('DBSCAN: силуэт не вычисляется (меньше двух кластеров или всё — шум)')

# 4.4 Elbow method: покажем WCSS для k=2..9
wcss = []  # суммарные внутрикластерные квадраты расстояний
for k in range(2, 10):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    wcss.append((k, km.inertia_))  # inertia_ = WCSS
print('Elbow WCSS (k, WCSS):', wcss)

# 4.5 Silhouette для разных k (2..9)
sil_scores = []
for k in range(2, 10):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels)
    sil_scores.append((k, sil))
print('Silhouette (k, score):', sil_scores)


KMeans: silhouette (k=3) = 0.8480303059596955
Agglomerative: silhouette (k=3) = 0.8480303059596955
DBSCAN: silhouette = 0.7823715398620228
Elbow WCSS (k, WCSS): [(2, 5763.464789461435), (3, 566.8595511244131), (4, 496.4279529785782), (5, 427.1286535498067), (6, 375.0331583497969), (7, 308.19836610418247), (8, 272.405731498748), (9, 234.28072349591127)]
Silhouette (k, score): [(2, 0.7049437310743717), (3, 0.8480303059596955), (4, 0.6636976714243523), (5, 0.49012744554094295), (6, 0.5168058221331912), (7, 0.358030586983425), (8, 0.3625632360122803), (9, 0.37134027555333055)]


### Задания (Кластеризация)
1. Попробуйте увеличить `cluster_std` при генерации данных с 1.0 до 1.8 — как меняются силуэты?  
2. В `DBSCAN` варьируйте `eps` (0.5, 0.7, 1.0, 1.2). Как меняется число кластеров и доля шума?  
3. Посмотрите WCSS и Silhouette для `k=2..9` и выберите разумное k. Обоснуйте.  
4. Замените `AgglomerativeClustering` на метод с `linkage='average'` (через `AgglomerativeClustering(linkage='average', ...)`) и сравните силуэт.  
5. Нормализуйте данные (например, `StandardScaler`) перед кластеризацией и сравните результаты для k-means.

## 5. Бустинг и ансамбли: идея и пример оттока
**Идея ансамблей:**
- **Бэггинг**: много однотипных моделей на разных подвыборках (пример: Random Forest). Снижаем дисперсию.
- **Бустинг**: модели строятся последовательно, каждая исправляет ошибки предыдущих (Gradient Boosting, XGBoost, LightGBM, CatBoost). Снижаем смещение.
- **Стэкинг**: предсказания базовых моделей идут в метамодель.

Ниже — пример бинарной задачи **оттока** на синтетических данных. Сравним базовую Logistic Regression и бустинг (`GradientBoostingClassifier`).

Отдельной ячейкой покажу, как аккуратно попробовать XGBoost/LightGBM/CatBoost, **если они установлены** (иначе — пропускаем).

In [8]:
from sklearn.ensemble import GradientBoostingClassifier  # бустинг из sklearn

rng = np.random.default_rng(42)     # генератор случайностей
n = 2000                            # количество клиентов

# синтетический датасет клиентов с простыми признаками
churn_df = pd.DataFrame({
    'age': rng.integers(18, 75, n),           # возраст
    'balance': rng.integers(0, 300000, n),    # баланс на счёте
    'transactions': rng.integers(1, 150, n),  # число операций
    'is_active': rng.integers(0, 2, n),       # активен (0/1)
    'tenure': rng.integers(1, 15, n)          # стаж (лет)
})

# формируем вероятность оттока как простую функцию признаков + шум
prob = (
    0.45 * (churn_df['is_active'] == 0).astype(float) +      # неактивные чаще уходят
    0.25 * (churn_df['transactions'] < 10).astype(float) +   # мало операций — риск выше
    0.20 * (churn_df['balance'] < 20000).astype(float) +     # низкий баланс — риск
    0.10 * (churn_df['tenure'] < 3).astype(float) +          # маленький стаж — риск
    rng.random(n) * 0.2                                      # добавим шум
)
y_churn = (prob > 0.5).astype(int)  # бинаризуем вероятность оттока порогом 0.5

Xc = churn_df  # признаки (таблица)
yc = y_churn   # целевая переменная (0/1)

# делим на train/test со стратификацией по целевой
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    Xc, yc, test_size=0.2, random_state=42, stratify=yc
)

# БАЗА: логистическая регрессия (часто baseline для бинарных задач)
base_lr = Pipeline(steps=[
    ('scaler', StandardScaler()),                           # масштабируем признаки
    ('clf', LogisticRegression(max_iter=1000, random_state=42))  # обучаем LR
])
base_lr.fit(Xc_train, yc_train)                   # обучение baseline
pred_lr = base_lr.predict(Xc_test)                # предсказания классов
proba_lr = base_lr.predict_proba(Xc_test)[:, 1]   # вероятности класса 1

print('--- Baseline: Logistic Regression ---')
print('Accuracy:', accuracy_score(yc_test, pred_lr))
print('Precision:', precision_score(yc_test, pred_lr))
print('Recall:', recall_score(yc_test, pred_lr))
print('F1:', f1_score(yc_test, pred_lr))
print('ROC-AUC:', roc_auc_score(yc_test, proba_lr))

# БУСТИНГ: GradientBoostingClassifier (скоро и сравнение)
gb = GradientBoostingClassifier(random_state=42)  # стандартные параметры — уже сильная модель
gb.fit(Xc_train, yc_train)                        # обучение бустинга
pred_gb = gb.predict(Xc_test)                     # предсказания классов
proba_gb = gb.predict_proba(Xc_test)[:, 1]        # вероятности класса 1

print('\n--- Gradient Boosting (sklearn) ---')
print('Accuracy:', accuracy_score(yc_test, pred_gb))
print('Precision:', precision_score(yc_test, pred_gb))
print('Recall:', recall_score(yc_test, pred_gb))
print('F1:', f1_score(yc_test, pred_gb))
print('ROC-AUC:', roc_auc_score(yc_test, proba_gb))


--- Baseline: Logistic Regression ---
Accuracy: 0.9025
Precision: 0.8106796116504854
Recall: 1.0
F1: 0.8954423592493298
ROC-AUC: 0.9252910488036801

--- Gradient Boosting (sklearn) ---
Accuracy: 0.895
Precision: 0.8078817733990148
Recall: 0.9820359281437125
F1: 0.8864864864864865
ROC-AUC: 0.9366888540515536


### (Опционально) XGBoost / LightGBM / CatBoost
Ниже — безопасная проверка: если библиотека установлена, модель обучится; если нет, выведем сообщение и продолжим работу ноутбука.  
⚠️ Установка при необходимости: `pip install xgboost lightgbm catboost`

In [9]:
try:
    from xgboost import XGBClassifier  # импортируем класс XGBoost
    xgb = XGBClassifier(
        n_estimators=300,        # число деревьев
        max_depth=4,            # глубина деревьев
        learning_rate=0.1,      # скорость обучения (шаг градиента)
        subsample=0.8,          # доля выборки для каждого дерева (bagging по объектам)
        colsample_bytree=0.8,   # доля фичей для каждого дерева (bagging по признакам)
        random_state=42,        # воспроизводимость
        eval_metric='logloss'   # метрика обучения внутри XGB
    )
    xgb.fit(Xc_train, yc_train)               # обучение XGBoost
    pred_xgb = xgb.predict(Xc_test)           # предсказания классов
    proba_xgb = xgb.predict_proba(Xc_test)[:, 1]  # вероятности класса 1
    print('--- XGBoost ---')
    print('Accuracy:', accuracy_score(yc_test, pred_xgb))
    print('Precision:', precision_score(yc_test, pred_xgb))
    print('Recall:', recall_score(yc_test, pred_xgb))
    print('F1:', f1_score(yc_test, pred_xgb))
    print('ROC-AUC:', roc_auc_score(yc_test, proba_xgb))
except Exception as e:
    print('XGBoost недоступен или не установлен:', e)

try:
    from lightgbm import LGBMClassifier  # импорт LightGBM
    lgbm = LGBMClassifier(random_state=42)  # базовая конфигурация
    lgbm.fit(Xc_train, yc_train)           # обучение LightGBM
    pred_lgbm = lgbm.predict(Xc_test)      # предсказания классов
    proba_lgbm = lgbm.predict_proba(Xc_test)[:, 1]  # вероятности класса 1
    print('\n--- LightGBM ---')
    print('Accuracy:', accuracy_score(yc_test, pred_lgbm))
    print('Precision:', precision_score(yc_test, pred_lgbm))
    print('Recall:', recall_score(yc_test, pred_lgbm))
    print('F1:', f1_score(yc_test, pred_lgbm))
    print('ROC-AUC:', roc_auc_score(yc_test, proba_lgbm))
except Exception as e:
    print('LightGBM недоступен или не установлен:', e)

try:
    from catboost import CatBoostClassifier  # импорт CatBoost
    cat = CatBoostClassifier(verbose=0, random_state=42)  # отключаем подробные логи
    cat.fit(Xc_train, yc_train)              # обучение CatBoost
    pred_cat = cat.predict(Xc_test)          # предсказания классов (массив строк '0'/'1' или чисел)
    # у catboost predict может вернуть строки; приведём к int
    pred_cat = pred_cat.astype(int).ravel()  # преобразуем к int и выровняем форму
    proba_cat = cat.predict_proba(Xc_test)[:, 1]  # вероятности класса 1
    print('\n--- CatBoost ---')
    print('Accuracy:', accuracy_score(yc_test, pred_cat))
    print('Precision:', precision_score(yc_test, pred_cat))
    print('Recall:', recall_score(yc_test, pred_cat))
    print('F1:', f1_score(yc_test, pred_cat))
    print('ROC-AUC:', roc_auc_score(yc_test, proba_cat))
except Exception as e:
    print('CatBoost недоступен или не установлен:', e)


--- XGBoost ---
Accuracy: 0.86
Precision: 0.8032786885245902
Recall: 0.8802395209580839
F1: 0.84
ROC-AUC: 0.9287348050679756
[LightGBM] [Info] Number of positive: 669, number of negative: 931
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000506 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 480
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.418125 -> initscore=-0.330475
[LightGBM] [Info] Start training from score -0.330475
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

### Задания (Бустинг и ансамбли)
1. В `GradientBoostingClassifier` измените `n_estimators` (100, 300, 600) и `learning_rate` (0.05, 0.1, 0.2). Как меняется ROC-AUC?  
2. Для XGBoost (если установлен) попробуйте `max_depth` = 3 и 6; сравните Recall и Precision.  
3. Постройте важности признаков (feature importance) для `RandomForest` и (если есть) XGBoost. Совпадают ли топ-фичи?  
4. Измените генерацию `prob` так, чтобы более важным стал `balance` и менее — `is_active`. Как изменятся метрики моделей?  
5. Попробуйте `class_weight='balanced'` в Logistic Regression на задаче оттока и сравните с бустингом.

# Итоговые упражнения (сквозные)
1. Возьмите один свой датасет (табличный), пройдите весь цикл: EDA → препроцессинг → train/test → модель (регрессия или классификация) → метрики → интерпретация.  
2. Сравните минимум три модели на одной задаче (например, LR, RF, GB). Сформируйте табличку с метриками.  
3. Для кластеризации: примените k-means и DBSCAN к тем же данным (после нормализации) — попробуйте объяснить бизнес-смысл кластеров.  
4. Напишите небольшой отчёт (1–2 страницы): цель, данные, подход, результаты, ограничения, планы улучшений.  
5. Вынесите лучший пайплайн в функцию/класс, чтобы переиспользовать на других данных.